### Further model training
This notebook contains training of the fine-tunned LlaMa3 model for medical assistant for final project of the Qualentum AI Solutions Development bootcamp.
We have already trained the model using Llama3 and ruslanmv/ai-medical-chatbot dataset (5000 rows). Now this model will be tunned with another dataset of medical supplies and their uses which is relevant for the project purpose.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-3-8b-medical-assistant/transformers/default/1/llama-3-8b-medical-assistant/model.safetensors.index.json
/kaggle/input/llama-3-8b-medical-assistant/transformers/default/1/llama-3-8b-medical-assistant/model-00003-of-00004.safetensors
/kaggle/input/llama-3-8b-medical-assistant/transformers/default/1/llama-3-8b-medical-assistant/config.json
/kaggle/input/llama-3-8b-medical-assistant/transformers/default/1/llama-3-8b-medical-assistant/model-00001-of-00004.safetensors
/kaggle/input/llama-3-8b-medical-assistant/transformers/default/1/llama-3-8b-medical-assistant/README.md
/kaggle/input/llama-3-8b-medical-assistant/transformers/default/1/llama-3-8b-medical-assistant/tokenizer.json
/kaggle/input/llama-3-8b-medical-assistant/transformers/default/1/llama-3-8b-medical-assistant/tokenizer_config.json
/kaggle/input/llama-3-8b-medical-assistant/transformers/default/1/llama-3-8b-medical-assistant/model-00004-of-00004.safetensors
/kaggle/input/llama-3-8b-medical-assistant/transforme

In [2]:
%pip install -U transformers 
%pip install -U datasets 
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install -U bitsandbytes 
%pip install -U wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 68.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.3
    Uninstalling transformers-4.42.3:
      Successfully uninstalled transformers-4.42.3
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 6.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 6.0 MB/s e

Importing the necessary Python packages for loading the dataset, model, and tokenizer and fine-tuning.

In [3]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

2024-07-29 19:09:20.809943: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-29 19:09:20.810085: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-29 19:09:20.931097: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("huggingface_token")

login(token = hf_token, add_to_git_credential=True)

wb_token = user_secrets.get_secret("wandb")

wandb.login(key=wb_token)
run = wandb.init(
    project='Medical assistant.',
    job_type="training", 
    anonymous="allow"
)

Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: v-mcsimov (v-mcsimov-universidad-de-la-rioja). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.17.5
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240729_190934-flz7seda
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run solar-sea-21
wandb: ⭐️ View project at https://wandb.ai/v-mcsimov-universidad-de-la-rioja/Medical%20assistant.
wandb: 🚀 View run at https://wandb.ai/v-mcsimov-universidad-de-la-rioja/Medical%20assistant./runs/flz7seda


Setting the base model, dataset, and new model variables.

In [5]:
base_model = "/kaggle/input/llama-3-8b-medical-assistant/transformers/default/1/llama-3-8b-medical-assistant"
dataset_name = "/kaggle/input/medicine-clean/df_medicine_clean.csv"
new_model = "llama-3-8b-medical-assistant-v2"

Setting the data type and attention implementation.

In [6]:
torch_dtype = torch.float16
attn_implementation = "eager"

Loading the model using 4-bit precision to reduce memory usage and speed up the fine-tuning process.

In [7]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Load tokenizer.

In [8]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
model, tokenizer = setup_chat_format(model, tokenizer)

Adding the adapter to the layer to improve the training time and make the process more memory-efficient.

In [9]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

Model description function.

In [10]:
def describe_model(model: AutoModelForCausalLM):
    n_params = model.num_parameters()
    print(f" - The model has {n_params/1e6:,.1f} millions parameters")
    print(f" - Architecture: {model.config.architectures[0]}")
    try:
        ctx = model.config.max_position_embeddings
    except:
        ctx = model.config.seq_length
    print(f" - Maximum context: {ctx:,} tokens")
    print(f" - Parameters format: {model.config.torch_dtype}")
    print(f" - Vocabulary size: {model.config.vocab_size:,} tokens")

describe_model(model)

 - The model has 8,072.2 millions parameters
 - Architecture: LlamaForCausalLM
 - Maximum context: 8,192 tokens
 - Parameters format: torch.float16
 - Vocabulary size: 128,258 tokens


### Loading the dataset.

In [11]:
#Importing the dataset
dataset = pd.read_csv(dataset_name)

In [12]:
dataset

,Excellent Review %,Average Review %,Poor Review %,Doctor,Patient
0,22,56,22,Medicine Name: Avastin 400mg Injection. Compos...,What is this medicine? How is it used?
1,47,35,18,Medicine Name: Augmentin 625 Duo Tablet. Compo...,What is this medicine? How is it used?
2,39,40,21,Medicine Name: Azithral 500 Tablet. Compositio...,What is this medicine? How is it used?
3,24,41,35,Medicine Name: Ascoril LS Syrup. Composition: ...,What is this medicine? How is it used?
4,34,37,29,Medicine Name: Aciloc 150 Tablet. Composition:...,What is this medicine? How is it used?
...,...,...,...,...,...
11493,24,19,57,Medicine Name: Zenegra Lido Spray. Composition...,What is this medicine? How is it used?
11494,67,33,0,Medicine Name: Zilarta-CT 40/6.25 Tablet. Comp...,What is this medicine? How is it used?
11495,0,80,20,Medicine Name: Zipcoz Tablet. Composition: Chr...,What is this medicine? How is it used?
11496,25,75,0,Medicine Name: Zestasil 100 Tablet. Compositio...,What is this medicine? How is it used?


Converting Pandas Dataframe into HuggingFace Dataset.

In [13]:
from datasets import Dataset

dataset = Dataset.from_pandas(dataset)

def format_chat_template(row):
    row_json = [{"role": "user", "content": row["Patient"]},
               {"role": "assistant", "content": row["Doctor"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc=4,
)

dataset['text'][42]

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/11498 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


"<|im_start|>user\nWhat is this medicine? How is it used?<|im_end|>\n<|im_start|>assistant\nMedicine Name: Avil Injection. Composition: Pheniramine (22.75mg). Uses: Treatment of Allergic conditionsTreatment of Respiratory disease with excessive mucusTreatment of Skin conditions with inflammation & itchingTreatment and prevention of Meniere's disease. Side effects: Sedation<|im_end|>\n"

Splitting the dataset into a training and validation set.

In [14]:
dataset = dataset.train_test_split(test_size=0.1)

In [15]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Excellent Review %', 'Average Review %', 'Poor Review %', 'Doctor', 'Patient', 'text'],
        num_rows: 10348
    })
    test: Dataset({
        features: ['Excellent Review %', 'Average Review %', 'Poor Review %', 'Doctor', 'Patient', 'text'],
        num_rows: 1150
    })
})

### Complaining and training the model.
Setting the model hyperparameters so that we can run it on the Kaggle.
Fine-tuning the model for one epoch and logging the metrics using the Weights and Biases.

In [16]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Setting up a supervised fine-tuning (SFT) trainer and provide a train and evaluation dataset, LoRA configuration, training argument, tokenizer, and model. We are keeping the max_seq_length to 512 to avoid exceeding GPU memory during training.

In [17]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will over

Map:   0%|          | 0/10348 [00:00<?, ? examples/s]

Map:   0%|          | 0/1150 [00:00<?, ? examples/s]

In [18]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
1035,0.631000,0.586852
2070,0.310600,0.475693
3105,0.362800,0.408352
4140,0.293600,0.361916


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


TrainOutput(global_step=5174, training_loss=0.5002314245172067, metrics={'train_runtime': 11170.7614, 'train_samples_per_second': 0.926, 'train_steps_per_second': 0.463, 'total_flos': 4.051698813456384e+16, 'train_loss': 0.5002314245172067, 'epoch': 1.0})

#### Model evaluation.

The Weights & Biases generates the run history and summary.

In [19]:
wandb.finish()
model.config.use_cache = True

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:               eval/loss █▅▂▁
wandb:            eval/runtime █▁▁▄
wandb: eval/samples_per_second ▁█▆▆
wandb:   eval/steps_per_second ▁█▆▆
wandb:             train/epoch ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb:       train/global_step ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
wandb:         train/grad_norm ▆▅▅▅▇▆▇▃▆▄▄▄▅▃▆▇▂█▅▅▂▄▂▄▁▄▅▄▃▁▁▅▁▄▃▂▃▅▃█
wandb:     train/learning_rate ███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:              train/loss ▆█▆▅▅▄▇▃▃▄▂▄▆▃▃█▁▃▃▃▂▁▁▄▁▂▃▃▃▁▁▃▁▂▂▂▂▃▂▆
wandb: 
wandb: Run summary:
wandb:                eval/loss 0.36192
wandb:             eval/runtime 479.4529
wandb:  eval/samples_per_second 2.399
wandb:    eval/steps_per_second 2.399
wandb:               total_flos 4.051698813456384e+16
wandb:              train/epoch 1.0
wandb:        train/global_step 5174
wandb:          train/grad_norm 1.91329
wandb:      train/learning_ra

In [20]:
messages = [
    {
        "role": "user",
        "content": "Hello doctor, I have bad acne. How do I get rid of it?"
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, 
                                       add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, 
                   truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=250, 
                         num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Medicine Name: Acneclear Gel. Composition: Clindamycin (1% w/w) + Nicotinamide (4% w/w). Uses: Treatment of Acne. Side effects: Skin peeling Dry skin Erythema skin redness Burning sensation at the site of application Skin irritation Itching Stinging sensation Skin rash Skin inflammation Indigestion Diarrhea Loss of appetite Headache Rash Vomiting Nausea Abdominal pain Flatulence Induced sweating Taste change Dizziness Nervousness Abnormal liver function tests Increased liver enzymes Application site redness Skin exfoliation Defeatism Depression Abnormality of voluntary movements Nasopharyngitis inflammation of the throat and nasal passages Upper abdominal pain Dyspepsia Loss of appetite Hypersensitivity Decreased appetite Abnormal liver function tests Increased lipase in the blood Elevated levels of serum amylase Inflammation of the nose Taste change Abnormal liver function tests Increased levels of serum glutamic oxaloacetic transaminase Increased levels of serum glutamic pyruvic tra

#### Saving the model file

In [21]:
trainer.model.save_pretrained(new_model)
trainer.model.push_to_hub(new_model, use_temp_dir=False)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/McSimoff/llama-3-8b-medical-assistant-v2/commit/27ae18ec29fc997da5f3fe93e3ed0561dd1477f5', commit_message='Upload model', commit_description='', oid='27ae18ec29fc997da5f3fe93e3ed0561dd1477f5', pr_url=None, pr_revision=None, pr_num=None)